# Setup

In [1]:
!pip install langchain_groq langchain_core langchain_huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 kB 8.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: requests-toolbelt
    Found existing installation: requests-toolbelt 0.10.1
    Uninstalling requests-toolbelt-0.10.1:
      Successfully uninstalled requests-toolbelt-0.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following 

In [2]:
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from langchain_groq import ChatGroq
from kaggle_secrets import UserSecretsClient

## Data Loading

In [3]:
data = pd.read_csv("/kaggle/input/tubes-nlp/seq2seq_data.csv")
data

,topic_category,original_text,base_word_text
0,9.0,what makes friendship click?,what make friendship click
1,2.0,why does zebras have stripes?,why zebra stripe
2,4.0,what did the itsy bitsy sipder climb up?,what itsy bitsy sipder climb up
3,4.0,what is the difference between a bachelors and...,what difference between bachelor and master de...
4,3.0,why do women get pms?,why woman get pm
...,...,...,...
174712,9.0,imperative: tell me what guys only guys must do!,tell me what guy only guy must
174713,9.0,tell me the story of any fantasy figure i'd ch...,tell me story of any fantasy figure i d choose
174714,8.0,imperative: reveal a secret about life.,reveal secret about life
174715,6.0,imperative: demande à domenech ce qu'il en est...,demande à domenech ce quil en est de son méti...


In [4]:
data["topic_category"] = data["topic_category"]-1
data["topic_category"]

0         8.0
1         1.0
2         3.0
3         3.0
4         2.0
         ... 
174712    8.0
174713    8.0
174714    7.0
174715    5.0
174716    4.0
Name: topic_category, Length: 174717, dtype: float64

# Data Preparation

## Data Cleaning

In [5]:
data.dropna(inplace=True)
data.isna().sum()

topic_category    0
original_text     0
base_word_text    0
dtype: int64

## Data Preprocessing

In [6]:
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

# Ensure you have the necessary NLTK data files
nltk.download('punkt')
nltk.download('stopwords')

# Define stopwords and punctuation
# stop_words = set(stopwords.words('english'))
stop_words = set()
stop_words.update(["imperative", "declarative"])
punctuation = string.punctuation

# Function to preprocess text
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', punctuation))
    # Tokenize text
    words = word_tokenize(text)
    # Remove stopwords
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

# Apply the preprocessing function to the 'original_text' column
data['processed_text'] = data['original_text'].apply(preprocess_text)
data[['original_text', 'processed_text']].head()

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,original_text,processed_text
0,what makes friendship click?,what makes friendship click
1,why does zebras have stripes?,why does zebras have stripes
2,what did the itsy bitsy sipder climb up?,what did the itsy bitsy sipder climb up
3,what is the difference between a bachelors and...,what is the difference between a bachelors and...
4,why do women get pms?,why do women get pms


## Data Splitting

In [7]:
test_ratio = 0.01  
instances_per_class = int(len(data) * test_ratio / 10)  # Calculate instances per class

# Sample data equally for each class
test_data = data.groupby('topic_category').sample(n=instances_per_class, random_state=42)

# Ensure balanced test set
print(test_data['topic_category'].value_counts())

topic_category
0.0    173
1.0    173
2.0    173
3.0    173
4.0    173
5.0    173
6.0    173
7.0    173
8.0    173
9.0    173
Name: count, dtype: int64


In [8]:
X_test = test_data['processed_text']
y_test = test_data['topic_category']

# Model Development

## QWEN

In [9]:
qwen_model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-1.5B-Instruct")
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-1.5B-Instruct")

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [10]:
# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the model to the GPU
qwen_model.to(device)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qw

In [11]:
def qwen_classifier(text:str) -> str:
    prompt = f"""
            You are a model that classify the topic from a text. 
            You only classify the text to 10 topics below.
            0: Society & Culture
            1: Science & Mathematics
            2: Health
            3: Education & Reference
            4: Computers & Internet
            5: Sports
            6: Business & Finance
            7: Entertainment & Music
            8: Family & Relationships
            9: Politics & Government

            Answer it in this format
            text: what did the itsy bitsy spider climb up
            topic: 3

            text: why do women get pms
            topic: 2

            text: {text}
            topic:
            """
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = qwen_model.generate(**inputs, max_new_tokens=20)
    generated_text = (tokenizer.decode(outputs[0], skip_special_tokens=True)).replace(prompt, "")
    return generated_text

In [12]:
print(qwen_classifier("tell me what guys only guys must do"))

 """
        else:
            print("I'm sorry, but I can't understand your message.")

   


## Llama

In [13]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
from langchain_core.messages import HumanMessage, SystemMessage

def format_prompt(text):
  messages = [
      SystemMessage(content="""You are a model that classify the topic from a text. 
            You only classify the text to 10 topics below.
            0: Society & Culture
            1: Science & Mathematics
            2: Health
            3: Education & Reference
            4: Computers & Internet
            5: Sports
            6: Business & Finance
            7: Entertainment & Music
            8: Family & Relationships
            9: Politics & Government

            Answer it in this format
            text: what did the itsy bitsy spider climb up
            topic: 3

            text: why do women get pms
            topic: 2
      
      """),
      HumanMessage(content=f"text: {text} topic:")
  ]
  return messages

In [15]:


chat_model = ChatGroq(
    api_key=  UserSecretsClient().get_secret("GROQ_API_KEY"),
    model="llama3-70b-8192",
    temperature=0,
    max_tokens=512,
)

In [16]:
from langchain_core.output_parsers import StrOutputParser

In [17]:
llama_classifier = (
    format_prompt
    | chat_model
    | StrOutputParser()
)

In [18]:
response = llama_classifier.invoke("tell me what guys only guys must do")

In [19]:
response.split("topic:")

['text: tell me what guys only guys must do\n', ' 8']

## Gemma

In [20]:
gemma_chat_model = ChatGroq(
    api_key=  UserSecretsClient().get_secret("GROQ_API_KEY"),
    model="gemma2-9b-it",
    temperature=0,
    max_tokens=512,
)

In [21]:
gemma_classifier = (
    format_prompt
    | gemma_chat_model
    | StrOutputParser()
)

In [22]:
response = gemma_classifier.invoke("tell me what guys only guys must do")

In [23]:
response.split("topic:")

['text: tell me what guys only guys must do \n', ' 8 \n']

# Evaluation

## Inference

In [24]:
from tqdm import tqdm

### QWEN

In [25]:
qwen_predictions = []
for text in tqdm(X_test):
    qwen_predictions.append(qwen_classifier(text))
qwen_predictions

100%|██████████| 1730/1730 [22:46<00:00,  1.27it/s]


[' 0\n            """\n        self.topic = topic\n\n    def __str__(self):\n        return',
 ' """\n        else:\n            return f"Text {text} does not have any recognized topic."\n\n   ',
 ' 4\n\n            text: how many days in february\n            topic: 7\n       ',
 ' 7\n\n    """\n    # TODO: Fill this in.\n    pass\n\n\ndef predict_topic(text',
 ' 0\n    """\n    \n    def __init__(self, tokenizer):\n        self.tokenizer =',
 ' """\n    return topic\n\n\ndef extract_keywords(text):\n    keywords = []\n    for word in text.split',
 ' 7\n\n            text: how can i increase my height\n            topic:\n\n            """\n        return',
 ' """\n        self._topics = dict()\n\n    def _add_topic(self, text, topic):\n       ',
 ' """\n    def __init__(self, config):\n        super().__init__()\n        self.config = config',
 ' 0\n\n    """\n\n    def __init__(self):\n        self.topic = None\n\n\ndef predict',
 ' 0\n        """\n        # Your code here\n        

### Llama

In [31]:
llama_predictions = []
for text in tqdm(X_test):
    llama_predictions.append(llama_classifier.invoke(text))
llama_predictions

100%|██████████| 1730/1730 [1:05:05<00:00,  2.26s/it]


['topic: 9',
 'topic: 0',
 'topic: 7',
 'text: if life everafter is taken out of religious ideologywhat happens to amount of believers\ntopic: 0',
 'topic: 0',
 'topic: 0',
 'topic: 3',
 'topic: 1',
 'topic: 0',
 'topic: 0',
 'text: tell me the day god supposedly died\ntopic: 0',
 'topic: 0',
 'topic: 4',
 'topic: 9',
 'topic: 0',
 'topic: 0',
 'topic: 0',
 'topic: 1',
 'topic: 4',
 'text: how do u grade instinct intellect and intuition\ntopic: 3',
 'topic: 3',
 'topic: 0',
 'text: how many ppl use myspace\ntopic: 4',
 'topic: 0',
 'topic: 0',
 'topic: 0',
 'text: which group is the coolest african americans european americans or van halen\ntopic: 7',
 'topic: 0',
 'text: does anyone have any brand new toys that would like to donate\ntopic: 8',
 'text: is it right for the nativity story to label itself based on a true story in the movie trailer\ntopic: 7',
 'text: what is the dialect spoken in perugia italy\ntopic: 0',
 'topic: 9',
 'topic: 1',
 'topic: 0',
 'text: does anyone feel lik

### Gemma

In [34]:
gemma_predictions = []
for text in tqdm(X_test):
    gemma_predictions.append(gemma_classifier.invoke(text))
gemma_predictions

100%|██████████| 1730/1730 [1:03:37<00:00,  2.21s/it]


['text: attn christians bible 3year old slave girls ordered raped \ntopic: 0 \n',
 'text: share your concerns about islam \ntopic: 9 \n',
 'text: which sign is he \ntopic: 3 \n',
 'text: if life everafter is taken out of religious ideology, what happens to the amount of believers?\ntopic: 9 \n',
 'text: the truth lies in embracing both unity and diversity \ntopic: 0 \n',
 'text: a jive turkey is a person imitating a wild turkey often done on thanksgiving day \ntopic: 7 \n',
 'text: help translating from english to latin the phrase is the prince is giving a ball \ntopic: 3 \n',
 'text: global warming is a serious environmental concern \ntopic: 1 \n',
 "text: most embarrassing thing you've seen someone else do \ntopic: 0 \n",
 'text: tell me people who arent racist regardless of psychologist opinions \ntopic: 0 \n',
 'text: tell me the day god supposedly died \ntopic: 9 \n',
 'text: could you tell me clever answers to any kind of insult without swear words \ntopic: 3 \n',
 'text: tell me

## Output Parsing

In [27]:
existing_labels = [str(label) for label in range(10)]
def parse_output_to_label(output: str) -> int:
    output_label = output.split("topic:")
    if (len(output_label) > 1):
        output_label = output_label[1]
    else:
        output_label = output_label[0]
        
    for label in existing_labels:
        if label in output_label:
            return int(label)
    return 0

### Qwen

In [28]:
qwen_label_predictions = [parse_output_to_label(output) for output in qwen_predictions]
qwen_label_predictions

[0,
 0,
 7,
 7,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 5,
 1,
 0,
 0,
 1,
 0,
 4,
 0,
 7,
 0,
 0,
 0,
 4,
 6,
 7,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 7,
 1,
 7,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 0,
 7,
 7,
 6,
 1,
 1,
 0,
 7,
 7,
 0,
 0,
 0,
 1,
 4,
 4,
 0,
 2,
 3,
 5,
 5,
 0,
 0,
 5,
 0,
 6,
 0,
 5,
 6,
 0,
 6,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 7,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 8,
 0,
 7,
 0,
 7,
 4,
 6,
 6,
 0,
 5,
 7,
 0,
 0,
 0,
 5,
 7,
 0,
 0,
 4,
 3,
 0,
 0,
 0,
 7,
 0,
 0,
 7,
 6,
 1,
 6,
 5,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 0,
 0,
 2,
 6,
 4,
 0,
 0,
 1,
 0,
 4,
 0,
 0,
 6,
 6,
 4,
 0,
 4,
 6,
 0,
 0,
 1,
 6,
 1,
 0,
 5,
 0,
 0,
 0,
 0,
 0,
 0,
 7,
 0,
 1,
 0,
 7,
 6,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 6,
 0,
 1,
 0,
 0,
 0,
 6,
 0,
 1,
 0,
 0,
 1,
 6,
 0,
 0,
 0,
 7,
 1,
 5,
 4,
 4,
 4,
 6,
 5,
 0,
 4,
 1,
 6,
 1,
 1,
 5,
 0,
 1,
 0,
 0,
 1,
 4,
 0,
 0,
 0,
 0,
 6,
 7,
 7,
 7,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 4,
 6,
 0,
 0,
 6,
 6,
 5,
 7,
 4,
 7,
 0,
 0,
 0,
 0,
 5,
 6,


### LLama

In [32]:
llama_label_predictions = [parse_output_to_label(output) for output in llama_predictions]
llama_label_predictions

[9,
 0,
 7,
 0,
 0,
 0,
 3,
 1,
 0,
 0,
 0,
 0,
 4,
 9,
 0,
 0,
 0,
 1,
 4,
 3,
 3,
 0,
 4,
 0,
 0,
 0,
 7,
 0,
 8,
 7,
 0,
 9,
 1,
 0,
 0,
 0,
 9,
 0,
 1,
 0,
 8,
 0,
 8,
 0,
 8,
 2,
 0,
 0,
 0,
 0,
 0,
 7,
 1,
 7,
 0,
 0,
 0,
 0,
 7,
 7,
 8,
 3,
 0,
 7,
 0,
 6,
 9,
 1,
 9,
 0,
 0,
 3,
 0,
 6,
 0,
 0,
 0,
 0,
 7,
 3,
 0,
 0,
 8,
 1,
 9,
 0,
 9,
 8,
 9,
 0,
 8,
 7,
 8,
 0,
 7,
 0,
 7,
 0,
 2,
 0,
 0,
 7,
 6,
 7,
 0,
 0,
 7,
 2,
 0,
 0,
 0,
 0,
 4,
 9,
 0,
 9,
 0,
 0,
 8,
 0,
 8,
 0,
 0,
 0,
 1,
 4,
 0,
 9,
 0,
 0,
 0,
 0,
 3,
 0,
 8,
 0,
 3,
 7,
 7,
 0,
 3,
 6,
 0,
 1,
 3,
 0,
 1,
 7,
 0,
 3,
 3,
 0,
 1,
 0,
 0,
 0,
 8,
 2,
 0,
 8,
 3,
 8,
 1,
 8,
 0,
 0,
 1,
 7,
 1,
 7,
 3,
 7,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 3,
 7,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 4,
 2,
 3,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 7,
 1,
 2,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 7,
 2,
 1,
 1,
 1,
 5,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 0,
 1,
 2,
 1,
 0,
 1,
 1,


### Gemma

In [35]:
gemma_label_predictions = [parse_output_to_label(output) for output in gemma_predictions]
gemma_label_predictions

[0,
 9,
 3,
 9,
 0,
 7,
 3,
 1,
 0,
 0,
 9,
 3,
 0,
 9,
 0,
 0,
 0,
 9,
 4,
 3,
 3,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 9,
 2,
 3,
 0,
 8,
 9,
 0,
 1,
 9,
 8,
 0,
 8,
 5,
 0,
 2,
 3,
 0,
 0,
 0,
 9,
 7,
 1,
 8,
 0,
 8,
 0,
 3,
 7,
 9,
 3,
 3,
 0,
 3,
 0,
 6,
 9,
 1,
 9,
 3,
 8,
 3,
 9,
 0,
 0,
 0,
 9,
 0,
 7,
 3,
 9,
 3,
 8,
 1,
 3,
 0,
 0,
 8,
 9,
 0,
 8,
 8,
 8,
 8,
 3,
 2,
 7,
 9,
 0,
 3,
 0,
 0,
 3,
 3,
 0,
 3,
 3,
 8,
 8,
 0,
 0,
 0,
 4,
 9,
 0,
 3,
 0,
 3,
 4,
 0,
 8,
 3,
 0,
 8,
 1,
 9,
 0,
 3,
 0,
 0,
 9,
 0,
 4,
 0,
 8,
 0,
 3,
 0,
 0,
 9,
 3,
 9,
 3,
 1,
 3,
 0,
 1,
 8,
 0,
 3,
 3,
 3,
 1,
 8,
 8,
 0,
 3,
 8,
 3,
 8,
 3,
 0,
 9,
 8,
 0,
 0,
 3,
 0,
 1,
 8,
 8,
 7,
 0,
 1,
 0,
 1,
 1,
 1,
 3,
 1,
 3,
 1,
 3,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 0,
 6,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 7,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 4,
 1,
 1,
 1,
 7,
 1,
 1,
 1,
 1,
 5,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 3,
 3,
 1,
 0,
 1,
 1,
 1,
 3,
 1,
 1,


## Metric Evaluation

In [29]:
from sklearn.metrics import classification_report

### Qwen

In [30]:
print("Classification Report:\n", classification_report(y_test, qwen_label_predictions))

Classification Report:
               precision    recall  f1-score   support

         0.0       0.11      0.54      0.18       173
         1.0       0.14      0.16      0.15       173
         2.0       0.20      0.03      0.06       173
         3.0       0.11      0.01      0.02       173
         4.0       0.13      0.14      0.14       173
         5.0       0.10      0.08      0.09       173
         6.0       0.09      0.08      0.08       173
         7.0       0.08      0.07      0.07       173
         8.0       0.00      0.00      0.00       173
         9.0       0.00      0.00      0.00       173

    accuracy                           0.11      1730
   macro avg       0.10      0.11      0.08      1730
weighted avg       0.10      0.11      0.08      1730



### Llama

In [33]:
print("Classification Report:\n", classification_report(y_test, llama_label_predictions))

Classification Report:
               precision    recall  f1-score   support

         0.0       0.42      0.48      0.45       173
         1.0       0.64      0.75      0.69       173
         2.0       0.74      0.83      0.78       173
         3.0       0.52      0.36      0.43       173
         4.0       0.72      0.90      0.80       173
         5.0       0.91      0.77      0.83       173
         6.0       0.57      0.47      0.51       173
         7.0       0.61      0.75      0.67       173
         8.0       0.68      0.64      0.66       173
         9.0       0.77      0.60      0.67       173

    accuracy                           0.65      1730
   macro avg       0.66      0.65      0.65      1730
weighted avg       0.66      0.65      0.65      1730



### Gemma

In [36]:
print("Classification Report:\n", classification_report(y_test, gemma_label_predictions))

Classification Report:
               precision    recall  f1-score   support

         0.0       0.34      0.36      0.35       173
         1.0       0.65      0.74      0.69       173
         2.0       0.78      0.72      0.75       173
         3.0       0.36      0.47      0.41       173
         4.0       0.65      0.86      0.74       173
         5.0       0.93      0.65      0.77       173
         6.0       0.58      0.40      0.47       173
         7.0       0.74      0.57      0.64       173
         8.0       0.59      0.69      0.63       173
         9.0       0.67      0.62      0.64       173

    accuracy                           0.61      1730
   macro avg       0.63      0.61      0.61      1730
weighted avg       0.63      0.61      0.61      1730

